## Generate toy sequence and phenotype data

In [382]:
import numpy as np
import pandas as pd

np.random.seed(42)
aa = "ACDEFGHIKLMNPQRSTVWY"

aa_to_index = {k:i for i,k in enumerate(aa)}
index_to_aa = {i:k for i,k in enumerate(aa)}

# Troponin C
refseq = "MDDIYKAAVEQLTEEQKNEFKAAFDIFVLGAEDGCISTKELGKVMRMLGQNPTPEELQEMIDEVDEDGSGTVDFDEFLVMMVRCMKDDSKGKSEEELSDLFRMFDKNADGYIDLDELKIMLQATGETITEDDIEELMKDGDKNNDGRIDYDEFLEFMKGVE"
ref_onehot = np.identity(20)[[aa_to_index[x] for x in refseq],:]

### Generate variants table

In [384]:
# brute force (list all possible variants and sample) to avoid replicates
variants = []
for pos in range(len(refseq)):
    for alt_ in np.arange(1,20):
        alt = (aa_to_index[refseq[pos]]+alt_)%20
        variants.append((pos, 
                         aa_to_index[refseq[pos]], 
                         alt,
                         refseq[pos]+str(pos)+index_to_aa[alt]))
        
variants_idxs = np.random.choice(len(variants), size = 150, replace=False)
variants = [variants[i] for i in variants_idxs]

var_table = pd.DataFrame({'pos': [v[0] for v in variants], 
                          'ref': [v[1] for v in variants], 
                          'alt': [v[2] for v in variants], 
                          'id': [v[3] for v in variants]})

assert len(var_table.drop_duplicates(subset='id')) == 150

### Generate sequences table

In [385]:
seq_varpos = [np.random.choice(list(set(var_table['pos'])), 
                               size = n, replace=False) for n in seq_nvars]

seq_varids = []
for varpos in seq_varpos:
    varids = []
    for pos in varpos:
        varids.append(np.random.choice(var_table[var_table['pos']==pos]['id']))
    seq_varids.append('_'.join(sorted(varids)))

seq_table = pd.DataFrame({'n_vars':[0], 'v_ids':['']})
seq_table = pd.concat((seq_table, pd.DataFrame({'n_vars':seq_nvars, 'v_ids':seq_varids})))
seq_table = seq_table.drop_duplicates(subset='v_ids', ignore_index=True)
seq_table.index.name = 'seq_id'

In [386]:
seq_table

,n_vars,v_ids
seq_id,,
0,0,
1,3,F103A_G139C_S97Y
2,3,D98S_E14K_I147P
3,3,A122Q_I60Y_S68E
4,1,I127Q
...,...,...
184,3,C83D_D24M_D61C
185,2,G29D_L96T
186,2,K91C_Y4V


### Generate sequence data

In [392]:
seq_data = np.repeat(ref_onehot[np.newaxis, :, :], 
                     repeats = len(seq_table)+1, 
                     axis =0)

for seq_id, vids in enumerate(seq_table['v_ids']):
    if seq_id>0:
        for vid in vids.split('_'):
            ref, pos, alt = aa_to_index[vid[0]], \
                            int(vid[1:-1]), \
                            aa_to_index[vid[-1]]
            assert seq_data[seq_id, pos, ref] == 1
            seq_data[seq_id, pos, :] = 0
            seq_data[seq_id, pos, alt] = 1


#### Save tables and matrix data

In [398]:
var_table.to_parquet("data/toy_data/variants.parquet")
seq_table.to_parquet("data/toy_data/sequences.parquet")
np.save("data/toy_data/seq_data.npy", seq_data)